In [1]:
import os

In [2]:
%pwd

'/Users/rociocuatecontzi/Desktop/programming/electricity-forecast/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rociocuatecontzi/Desktop/programming/electricity-forecast'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PredictionConfig:
    root_dir: Path
    model_path: Path
    target_column: str
    columns: str

In [6]:
from eForecaster.constants import *
from eForecaster.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prediction_config(self) -> PredictionConfig:
        config = self.config.model_evaluation
        params = self.params.XGRBoost
        schema =  self.schema

        create_directories([config.root_dir])

        model_evaluation_config = PredictionConfig(
            root_dir=config.root_dir,
            model_path = config.model_path,
            target_column = schema.TARGET_COLUMN.name,
            columns = schema.COLUMNS,
        )
        return model_evaluation_config


In [8]:
import os
import pandas as pd
import numpy as np
import joblib
from datetime import datetime

In [54]:

class Prediction:
    def __init__(self, config: PredictionConfig):
        self.config = config

    
    def get_model(self):
        model = joblib.load(self.config.model_path)
        return model
    
    def create_datetime_df(self, start_date: str, end_date: str):
        start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
        end_date = datetime.strptime(end_date,"%Y-%m-%d %H:%M:%S")
        datetime_df = pd.date_range(start_date, end_date, freq='15min')
        datetime_df = pd.DataFrame(datetime_df.to_frame(index=False, name="datetime"))
        return datetime_df

    def create_prediction_df(self, df):
        df.datetime.describe()
        df['hour'] = df.datetime.dt.hour
        df['dayofweek'] = df.datetime.dt.dayofweek
        df['quarter'] = df.datetime.dt.quarter
        df['month'] = df.datetime.dt.month
        df['year'] = df.datetime.dt.year
        df['dayofyear'] = df.datetime.dt.dayofyear
        df['minute'] = df.datetime.dt.minute
        return df
        


In [55]:
try:
    config = ConfigurationManager()
    prediction_config = config.get_prediction_config()
    prediction = Prediction(config=prediction_config)
    model = prediction.get_model()
    start_date = "2023-06-15 00:00:00"
    end_date = "2023-06-22 00:15:00"
    datetime_df = prediction.create_datetime_df(start_date, end_date)
    df = prediction.create_prediction_df(datetime_df)
    print(df)

except Exception as e:
    raise e

[2024-01-28 15:23:30,923: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-28 15:23:30,926: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-28 15:23:30,928: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-28 15:23:30,930: INFO: common: created directory at: artifacts]
[2024-01-28 15:23:30,932: INFO: common: created directory at: artifacts/model_evaluation]


AttributeError: 'Series' object has no attribute 'dayofweek'